# FIRST ATTEMPT TO CNN

The purpose of this first file is to implement a Convolutional Neural Network able to classify the images in the different classes with an accuracy of at least 30%.

Initially, the data must be read and pre-processed. In general it is necessary to convert the images to grayscale and resize the various images to 64x64 pixels.

## Libreries

List of list of libraries used

In [1]:
import os
import glob
import cv2 as cv2
import numpy as np
import keras as ks

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


import import_ipynb
import dataSetUtility as dsu

importing Jupyter notebook from dataSetUtility.ipynb


## DATA PREPROCESSING 

In the following cells we construct and prepare the data for CNN

In [2]:
#path to the directories
pathTrain=r'C:\Users\adria\Desktop\CVPR\ImageSet\train'
pathTest=r'C:\Users\adria\Desktop\CVPR\ImageSet\test'

labels = [os.path.basename(i) for i in glob.glob(pathTrain + '/*', recursive=True)]
numberOfClasses = len(labels)
print("Class number: ", numberOfClasses)
print("Class names: ", labels)

Class number:  15
Class names:  ['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']


In [3]:
xTrainRaw, yTrainRaw = dsu.load_images(pathTrain, labels)
xTest, yTest = dsu.load_images(pathTest, labels)

print('len train x: ',len(xTrainRaw))
print('len test x: ',len(xTest))
print('len train y: ',len(yTrainRaw))
print('len test y: ',len(yTest))

len train x:  1500
len test x:  2985
len train y:  1500
len test y:  2985


In [4]:
xTrainRaw = dsu.reshape(xTrainRaw, 64,64,1)
xTest = dsu.reshape(xTest,64,64,1)

In [5]:
yTrainCategorical = to_categorical(yTrainRaw)
yTestCategorical = to_categorical(yTest)

Split the training set into train set (85%) and validation set (15%)

In [6]:
xTrain, xValidation, yTrain, yValidation = train_test_split(xTrainRaw, yTrainCategorical, train_size=0.85, random_state=275)
print('len Train Set: ',len(xTrain))
print('len Validation Set: ',len(xValidation))
print('len Test Set: ',len(xTest))

len Train Set:  1275
len Validation Set:  225
len Test Set:  2985


In [7]:
cv2.imshow("Initial image", xTrainRaw[9])
cv2.waitKey(0)
cv2.destroyAllWindows()